# Word2Vec Training
We will be training multiple iterations of the word2vec model on our annotations file. These iterations include a trained word2vec for:
- The whole training set,
- One for each source in the training set,
- One for all sources dubbed as formal (i.e. TED),
- And one for all sources dubbed as informal (i.e. reddit).

We will be comparing the performance of each of these embeddings when paired with our supervised and unsupervised models.

## Dependencies
This section contains all imports and initialized global variables.

### Clone Github Repository
I am including this step to ensure usability in multiple environments (i.e. Google Colab and Great Lakes Cluster for this project).

In [1]:
!git clone https://github.com/d-atallah/implicit_gender_bias.git

fatal: destination path 'implicit_gender_bias' already exists and is not an empty directory.


### Import Libraries
From here we are importing all necessary libraries as well as a configuration file from our repository containing shared functions that we may use across our notebooks.

In [2]:
from implicit_gender_bias import config as cf
import pandas as pd
import numpy as np
import joblib
import os
from gensim.models import Word2Vec
from tqdm import tqdm

### Set Reoccuring Variables
Here I will set variables that will be used throughout this notebook. Filepath is specified based on environment (Colab vs. GLC). I am also specifying the exact files I want to split into train and test files from the raw RtGender sources. The extract_dfs function is dynamic, I can specify as many files that I want to split and load.

In [3]:
filepath = cf.filepath()
files = ['annotations']
df_dict = cf.extract_dfs(filepath, files)
raw_annotations = df_dict['annotations']
load_dict = cf.load_df(filepath, raw_annotations, 'annotations')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Begin Training
We will be training over each iteration of source and combination of sources. The data used to train and the models trained will be saved in the shared drive

In [4]:
df = pd.read_csv(load_dict['X_train']).iloc[:,1:]
df.head()

,index,source,post_text,response_text,sentiment,relevance,sourceID
0,13840,ted,Monica Lewinsky gave a talk about communicatio...,"A courageous speech, a very good talker and a ...",Mixed,Content,ted13840
1,573,facebook_wiki,Tennis + fitness + school = brain dead. Trying...,"Good luck for the season, Madison!",Positive,ContentPoster,facebook_wiki573
2,8213,fitocracy,Hey chervan--you must be motivated! You joined...,I agree...you're looking good!,Positive,ContentPoster,fitocracy8213
3,9092,fitocracy,"Thanks for the follow back, i'll be watching y...","Thank you too for following! The ""fighting gra...",Positive,Poster,fitocracy9092
4,14354,ted,"Louie Schwartzberg gave a talk about bacteria,...",Just Amazing!!!!!!!!!,Positive,Content,ted14354


In [5]:
src_lst = list(df.source.unique())
src_lst.extend(['formal', 'informal', 'all'])
rm_stop_param_lst = [True, False]
lemma_param_lst = [True, False]
print(src_lst)

['ted', 'facebook_wiki', 'fitocracy', 'facebook_congress', 'reddit', 'formal', 'informal', 'all']


In [6]:
train_dict = {}
# create paths if they do not already exist
if os.path.exists(filepath + 'trns/word2vec/') == False:
  os.mkdir(filepath + 'trns/word2vec/')
if os.path.exists(filepath + 'trns/word2vec/data/') == False:
  os.mkdir(filepath + 'trns/word2vec/data/')
if os.path.exists(filepath + 'trns/word2vec/models/') == False:
  os.mkdir(filepath + 'trns/word2vec/models/')
# iterate over each source and combination of preprocessing settings
for src in tqdm(src_lst):
  for stop_param in rm_stop_param_lst:
    for lemma_param in lemma_param_lst:
      name = 'word2vec/data/' + src + '_stop_param_' + str(stop_param) + \
        '_lemma_param_' + str(lemma_param)
      if src == 'formal': srcs = ['ted', 'facebook_congress']
      elif src == 'informal': srcs = ['fitocracy', 'facebook_wiki', 'reddit']
      elif src == 'all': srcs = list(df.source.unique())
      else: srcs = [src]
      # below function inits df AND saves to shared drive
      temp_df = cf.preprocess(filepath = load_dict['X_train'], name = name,
                              rm_stopwords = stop_param, lemmatize = lemma_param)
      temp_ser = temp_df[temp_df.source.isin(srcs)].processed_response
      # below are the parameters I found online to be recommended for short text
      temp_model = Word2Vec(temp_ser, vector_size = 100, window = 5,
                            min_count = 5, sg = 1, hs = 0, negative = 5,
                            sample = 1e-3, workers = 4, epochs = 10)
      name2 = 'word2vec/models/' + src + '_stop_param_' + str(stop_param) + \
        '_lemma_param_' + str(lemma_param)
      # saving model to shared location
      temp_model.save(filepath + 'trns/' + name2 + '.pkl')

  0%|          | 0/8 [00:00<?, ?it/s][nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 12%|█▎        | 1/8 [00:19<02:16, 19.47s/it][nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 25%|██▌       | 2/8 [00:24<01:05, 10.97s/it][nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 38%|███▊      | 3/8 [00:30<00:43,  8.72s/it][nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading